In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-10 21:35:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-04-10 21:35:26 (10.8 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [7]:
vine_review_df = df.select(["customer_id", "product_id", "product_title","review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_date"])
vine_review_df.show()

+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|product_id|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|   41409413|B00428R89M|yoomall 5M Antenn...|R2MTG1GCZLR2DK|          5|            0|          0|   N|                Y| 2015-08-31|
|   49668221|B000068O48|Hosa GPM-103 3.5m...|R2HBOEM8LE9928|          5|            0|          0|   N|                Y| 2015-08-31|
|   12338275|B000GGKOG8|Channel Master Ti...|R1P4RW1R9FDPEE|          5|            1|          1|   N|                Y| 2015-08-31|
|   38487968|B000NU4OTA|LIMTECH Wall char...|R1EBPM82ENI67M|          1|            0|          0|   N|                Y| 2015-08-31|
|   23732619|B00JOQIO6S|Skullcandy Air Ra...|R372S58V6D11AT|  

In [8]:
total_votes_df = vine_review_df.filter("total_votes>=20")
total_votes_df.show()

+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|product_id|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|   14380991|B00NG57H4S|Sony MDRZX110 Noi...|R1FBO737KD9F2N|          5|           19|         23|   N|                Y| 2015-08-31|
|   51242401|B00ICNXESC|GS Portalac PX120...|R227GSNWI6BSZV|          1|           20|         20|   N|                Y| 2015-08-31|
|    8464339|B00L108SAW|Nitecore D4 Smart...|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y| 2015-08-31|
|   49422039|B00870ZLJ0|AmazonBasics High...|R248FG65D76D5Y|          1|           42|         53|   N|                Y| 2015-08-31|
|   53003148|B00L2CK758|Kingzer Top Incep...|R3B6BXFKGW52SG|  

In [15]:
votes_50_df = total_votes_df.filter("helpful_votes/total_votes >= .50")
votes_50_df.show()

+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|product_id|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|   14380991|B00NG57H4S|Sony MDRZX110 Noi...|R1FBO737KD9F2N|          5|           19|         23|   N|                Y| 2015-08-31|
|   51242401|B00ICNXESC|GS Portalac PX120...|R227GSNWI6BSZV|          1|           20|         20|   N|                Y| 2015-08-31|
|    8464339|B00L108SAW|Nitecore D4 Smart...|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y| 2015-08-31|
|   49422039|B00870ZLJ0|AmazonBasics High...|R248FG65D76D5Y|          1|           42|         53|   N|                Y| 2015-08-31|
|   53003148|B00L2CK758|Kingzer Top Incep...|R3B6BXFKGW52SG|  

In [17]:
paid_vine_review_df = votes_50_df.filter("vine =='Y'")
paid_vine_review_df.show()

+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|product_id|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|   36356690|B0117RFX38|Bose SoundTrue Ar...|R184FOUNZZ7KO8|          5|           15|         20|   Y|                N| 2015-08-31|
|   37291590|B0117RGG8E|Bose SoundLink ar...| R82QWN2X2OCHB|          5|          176|        208|   Y|                N| 2015-08-30|
|   53030638|B00TP1C1UC|AmazonBasics 6-Ou...|R1UYHBYE6790BU|          5|           44|         53|   Y|                N| 2015-08-28|
|   50672825|B012B7S46Q|Bose Solo 15 Seri...|R2J3YLX1L4EH2B|          5|          299|        321|   Y|                N| 2015-08-26|
|   52563108|B00TTWZ9VU|Sony SRSX77 Power...|R3QDI539WTXKE2|  

In [21]:
unpaid_vine_review_df = votes_50_df.filter("vine == 'N'")
unpaid_vine_review_df.show()

+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|customer_id|product_id|       product_title|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_date|
+-----------+----------+--------------------+--------------+-----------+-------------+-----------+----+-----------------+-----------+
|   14380991|B00NG57H4S|Sony MDRZX110 Noi...|R1FBO737KD9F2N|          5|           19|         23|   N|                Y| 2015-08-31|
|   51242401|B00ICNXESC|GS Portalac PX120...|R227GSNWI6BSZV|          1|           20|         20|   N|                Y| 2015-08-31|
|    8464339|B00L108SAW|Nitecore D4 Smart...|R3SJTYZBYBG4EE|          4|           99|         99|   N|                Y| 2015-08-31|
|   49422039|B00870ZLJ0|AmazonBasics High...|R248FG65D76D5Y|          1|           42|         53|   N|                Y| 2015-08-31|
|   53003148|B00L2CK758|Kingzer Top Incep...|R3B6BXFKGW52SG|  

In [27]:
total_number_paid_reviews = paid_vine_review_df.count()
total_number_paid_reviews

1080

In [28]:
total_number_unpaid_reviews = unpaid_vine_review_df.count()
total_number_unpaid_reviews

49673

In [33]:
five_star_paid_reviews = paid_vine_review_df.filter(paid_vine_review_df.star_rating==5).count()
five_star_paid_reviews

454

In [34]:
five_star_unpaid_reviews = unpaid_vine_review_df.filter(unpaid_vine_review_df.star_rating==5).count()
five_star_unpaid_reviews

23043

In [35]:
percent_five_star_paid = (five_star_paid_reviews/total_number_paid_reviews)*100
percent_five_star_paid

42.03703703703704

In [36]:
percent_five_star_unpaid = (five_star_unpaid_reviews/total_number_unpaid_reviews)*100
percent_five_star_unpaid

46.3893865882874